1- Preprocessing :
define characters accepted and tashkeel accepted and remove from the training set any tashkeel and unwanted characters (eg. brackets, numbers... etc)

In [2]:
import re
import pyarabic.araby as araby

In [3]:
tashkeel = ["ْ", "ّ", "ٌ", "ٍ", "ِ", "ً", "َ", "ُ"]
arabicCharacters = "ىعظحرسيشضقثلصطكآماإهزءأفؤغجئدةخوبذتن"
punctuations = [".", "،", ":", "؛", "-", "؟"]
validCharacters= tashkeel + list(arabicCharacters) + punctuations
charcters_without_tashkeel = list(arabicCharacters) + punctuations

In [4]:
#reading the training dataset
f = open(r"train.txt", "r",encoding="utf-8").read()
print(type(f))

<class 'str'>


In [5]:
print(len(f))
#regex to keep arabic letters only and remove any other character (eg. brackets, numbers ...etc)
characters_regex =r'[\s\.\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+'
processedData = re.findall(characters_regex,f)
processedData = " ".join(processedData)
processedData = re.sub(r"\s+"," " ,processedData) #substitute many spaces with one space only
print(len(processedData))

18515271
17997403


In [6]:
w = open('processed.txt', 'w', encoding='utf-8')

w.write(processedData)
w.close()

In [7]:
#remove diacritics
without_diacritics= araby.strip_diacritics(processedData)
print(len(without_diacritics))
w2 = open('withoutDiacritics.txt', 'w', encoding='utf-8')

w2.write(without_diacritics)
w2.close()

10711561


In [8]:
#take the procrssed text and separate it into sentences
fileRead=open('processed.txt', encoding='utf-8')
lines=fileRead.readlines()
longString=' '.join(lines)
longStringSplited=re.sub(r"\n", ' ', longString)
#the dot itself got removed in the process
longStringSplited=longStringSplited.split('.')
print(len(longStringSplited))
w2 = open('withoutDiacritics.txt', 'w', encoding='utf-8')

w2.write(without_diacritics)

w3 = open('processed_sentences_separated.txt', 'w', encoding='utf-8')
for line in longStringSplited:
 w3.write(line)

 #now longStringSplited is the list of procrssed text without practices and numbers and dots 
 #the rest of the punctuation still there  


 


40836


In [33]:
# now we want to create a list of words and a crossbonding list of punctuation for each word 

#first remove all puncturation 

fileRead=open('processed_sentences_separated.txt', encoding='utf-8')
lines=fileRead.readlines()
longString=' '.join(lines)
longString=re.sub(r"\n", ' ', longString)
punctuations = ["،", ":", "؛", "-", "؟"]
for element in punctuations:
     longString=re.sub(element, '', longString)



In [44]:
# now the varable called longString has a single string with all the processed words in it 
listOfwordsWith_Diacritics=list()
listOfwordsWith_NoDiacritics=list()


without_diacritics= araby.strip_diacritics(longString)


without_diacritics=re.sub(r"\s+", ' ', without_diacritics)
listOfwordsWith_NoDiacritics=without_diacritics.split(" ")
print(listOfwordsWith_NoDiacritics[0:100])


listOfwordsWith_Diacritics=re.sub(r"\s+", ' ', longString)
listOfwordsWith_Diacritics=listOfwordsWith_Diacritics.split(" ")
print(listOfwordsWith_Diacritics[0:100])




['قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي', 'ابن', 'عرفة', 'قوله', 'بلفظ', 'يقتضيه', 'كإنكار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوبه', 'من', 'الدين', 'ضرورة', 'كإلقاء', 'مصحف', 'بقذر', 'وشد', 'زنار', 'ابن', 'عرفة', 'قول', 'ابن', 'شاس', 'أو', 'بفعل', 'يتضمنه', 'هو', 'كلبس', 'الزنار', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجاسة', 'والسجود', 'للصنم', 'ونحو', 'ذلك', 'وسحر', 'محمد', 'قول', 'مالك', 'وأصحابه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', 'هو', 'كالزنديق', 'إذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يستتب', 'قوله', 'لعدم', 'ما', 'تتعلق', 'إلخ', 'أي', 'الوصية', 'قوله', 'ما', 'مر', 'أي', 'قبيل', 'قول', 'المتن', 'لغت', 'ولو', 'اقتصر', 'على', 'أوصيت', 'له', 'بشاة', 'أو', 'أعطوه', 'شاة', 'ولا', 'غنم', 'له', 'عند', 'الموت', 'هل']
['قَوْلُهُ', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', 'قَالَ', 'الزَّرْكَشِيُّ', 'ابْنُ', 'عَرَفَةَ', 'قَوْلُهُ', 'بِلَفْظٍ', 'يَقْتَضِيه', 'كَإِنْكَارِ', 'غَيْرِ', 'حَدِيثٍ', 'بِالْإِسْلَامِ', 'وُ

In [49]:
#now that we have two separated lists we need to get the dicritics list 

print(len(listOfwordsWith_Diacritics))
print(len(listOfwordsWith_NoDiacritics))
listofDiacritrcs_Toword=list()
temp=list()
# for word in listOfwordsWith_Diacritics:
#     for char in word:
#         if char in arabicCharacters:
#             continue
#         else:
#             temp

2101285
2101285
